In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip3 install autogluon

In [ ]:
train_df = pd.read_pickle('../input/amex-data-pckl-files/train_agg.pkl', compression='gzip')
train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

In [ ]:
train_labels = train_labels.sort_values(by=['customer_ID']).drop('customer_ID', axis=1)
train_df = pd.concat([train_df, train_labels], axis=1)

In [ ]:
gc.collect()

In [ ]:
# COMPETITION METRIC FROM Konstantin Yakovlev
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
from autogluon.core.metrics import make_scorer
amex_scorer = make_scorer(name='amex_metric',
                          score_func=amex_metric_mod,
                          optimum=1,
                           greater_is_better=True)

In [ ]:


hyperparameters = {  # hyperparameters of each model type
                   'GBM': {},
                   'NN_TORCH': {},  # NOTE: comment this line out if you get errors on Mac OSX
                  }

In [ ]:
gc.collect()

In [ ]:
train_df.shape

In [ ]:
# for i in range(5):
#     if i != 4:
#         temp_df = train_df.sample(n=91782, random_state=42)
#     else:
#         temp_df = train_df
#     print(temp_df.shape)
#     print(temp_df['target'].value_counts())
#     train_df = train_df.drop(temp_df.index, axis=0)
#     # break

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label='target', path=save_path, eval_metric='recall').fit(train_df.drop('customer_ID', axis=1), 
                                                                                          hyperparameters=hyperparameters, 
                                                                                          auto_stack=True,
                                                                                           num_stack_levels=0
                                                                                          # time_limit=10*60,
                                                                                          #ag_args_fit={'num_gpus': 1}
                                                                                      )